# Aiida

## Define workflow with aiida

In [ ]:
from python_workflow_definition.aiida import write_workflow_json
from python_workflow_definition.shared import get_dict, get_list

from aiida import load_profile, orm

load_profile()

workflow_json_filename = "aiida_qe.json"

In [ ]:
from aiida_workgraph import task, WorkGraph

In [ ]:
from workflow import (
    generate_structures,
    get_bulk_structure,
    calculate_qe as _calculate_qe,
    plot_energy_volume_curve,
)

In [ ]:
calculate_qe = task(outputs=["energy", "volume", "structure"])(_calculate_qe)

In [ ]:
wg = WorkGraph("wg-qe")

### Prepare the inputs

In [ ]:
element = orm.Str("Al")
a = orm.Float(4.04)
cubic = orm.Bool(True)
relax_workdir = orm.Str("mini")
pseudopotentials = orm.Dict({"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"})
kpts = orm.List([3, 3, 3])
calc_type_relax = orm.Str("vc-relax")
calc_type_scf = orm.Str("scf")
smearing = orm.Float(0.02)
strain_lst = orm.List([0.9, 0.95, 1.0, 1.05, 1.1])

### Actual tasks to construct the EOS workflow

In [ ]:
get_bulk_structure_task = wg.add_task(
    get_bulk_structure,
    element=element,
    a=a,
    cubic=cubic,
)

In [ ]:
relax_prepare_input_dict_task = wg.add_task(
    get_dict,
    structure=get_bulk_structure_task.outputs.result,
    calculation=calc_type_relax,
    kpts=kpts,
    pseudopotentials=pseudopotentials,
    smearing=smearing,
)

relax_task = wg.add_task(
    calculate_qe,
    input_dict=relax_prepare_input_dict_task.outputs.result,
    working_directory=relax_workdir,
)

In [ ]:
generate_structures_task = wg.add_task(
    generate_structures,
    structure=relax_task.outputs.structure,
    strain_lst=strain_lst,
)

In [ ]:
get_volumes_task = wg.add_task(get_list)

In [ ]:
get_energies_task = wg.add_task(get_list)

In [ ]:
strain_dir_tasks, scf_qe_tasks, scf_get_dict_tasks = [], [], []

for i, strain in enumerate(strain_lst):

    structure_key = f"s_{i}"
    strain_dir = orm.Str(f"strain_{i}")
    generate_structures_task.add_output("workgraph.any", structure_key)

    scf_prepare_input_dict_task = wg.add_task(
        get_dict,
        structure=generate_structures_task.outputs[structure_key],
        calculation=calc_type_scf,
        kpts=kpts,
        pseudopotentials=pseudopotentials,
        smearing=smearing,
    )

    scf_qe_task = wg.add_task(
        calculate_qe,
        input_dict=scf_prepare_input_dict_task.outputs.result,
        working_directory=strain_dir,
    )

    # collect energy and volume
    get_energies_task.set({f"{i}": scf_qe_task.outputs.energy})
    get_volumes_task.set({f"{i}": scf_qe_task.outputs.volume})

In [ ]:
plot_energy_volume_curve_task = wg.add_task(
    plot_energy_volume_curve,
    volume_lst=get_volumes_task.outputs.result,
    energy_lst=get_energies_task.outputs.result,
)

In [ ]:
wg

In [ ]:
_ = write_workflow_json(wg=wg, file_name=workflow_json_filename)


In [ ]:
!cat {workflow_json_filename}

## Load Workflow with jobflow

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

In [ ]:
from jobflow.managers.local import run_locally

In [ ]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
flow[0].function_kwargs["a"] = 4.05

In [ ]:
result = run_locally(flow)
result

## Load Workflow with pyiron_base

In [ ]:
from python_workflow_definition.pyiron_base import load_workflow_json

In [ ]:
delayed_object_lst = load_workflow_json(file_name=workflow_json_filename)
delayed_object_lst[-1].draw()

In [ ]:
delayed_object_lst[0].input['a'] = 4.05

In [ ]:
delayed_object_lst[-1].pull()

## Load Workflow with pyiron_workflow

In [ ]:
from python_workflow_definition.pyiron_workflow import load_workflow_json

In [ ]:
wf = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
wf.get_bulk_structure.inputs.a.value = 4.05

In [ ]:
wf.draw(size=(10,10))

In [ ]:
wf.run()